In [1]:
import sys
import tweepy
import sys
import tweepy
from nltk.corpus import stopwords
import re
from collections import Counter
import pandas as pd
import unidecode

consumer_key='YJDELxapXCz0dIbfJcJ27sfU5'
consumer_secret='q5ucGBB9xU9p5Xvorp9UwtvN3qaEhOlOFye3bL83YRRE9riTc0'
access_key='959019738855862273-PqPmgzE4fIrlLb9x49xSUAN5HoqjS15'
access_secret='jtYyvfdPiXGoeuJ5QnmnaAluw8bSitinnqUMI4Sl2cDWm'

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


df = pd.read_csv('cities.csv', encoding='utf-8')

In [2]:
df.head()

,city,lat,lng,country,iso2,admin,capital,population,population_proper
0,Madrid,40.412752,-3.707721,Spain,ES,Madrid,primary,5567000.0,50437.0
1,Barcelona,41.398371,2.174100,Spain,ES,Catalonia,admin,4920000.0,1581595.0
2,Sevilla,37.382826,-5.973167,Spain,ES,Andalusia,admin,1212045.0,703021.0
3,Bilbao,43.260919,-2.938764,Spain,ES,Basque Country,minor,875552.0,353187.0
4,Valencia,39.466667,-0.366667,Spain,ES,Valencia,admin,808000.0,805304.0


In [67]:
Andalusia = ["Andalusia", "37.3399964,-4.5811614,250km"]
#Extremadura = 
Madrid = ["Madrid", "40.5248319,-3.7715628,60km"]
Catalonia = ["Catalonia", "41.8523094,1.5745043,150km"]
#CastillaLaMancha = 
#CastillaLeon = 
#Cantabria = 
Basque_Country = ["Basque Country", "42.9911816,-2.5543023,100km"]
#ComunidadValenciana = 
Canary_Islands = ["Canary Islands", "28.5306525,-15.7464439,400km"]
#Aragon = 
#LaRioja = 
#Navarra = 
#Asturias = 
#Murcia = 

CAS = [Andalusia, Madrid, Catalonia, Basque_Country, Canary_Islands]

In [68]:
#Rate limits are divided into 15 minute intervals. All endpoints require authentication, 
#so there is no concept of unauthenticated calls and rate limits. There are two initial buckets available 
#for GET requests: 15 calls every 15 minutes, and 180 calls every 15 minutes.

In [74]:
list(df[df["admin"] == "Canary Islands"]["city"])

['Santa Maria de Gracia',
 'Arrecife',
 'Las Palmas de Gran Canaria',
 'Santa Cruz de Tenerife']

In [75]:
CAS_ids = []
for CA in CAS:
    CA_cities = df[df["admin"] == CA[0]]["city"]
    tweets = tweepy.Cursor(api.search, geocode = CA[1]).items(300)  
    CA_ids = []
    for tweet in tweets:
        search_location = re.search('location\':(.+?),', str(tweet.user))
        if search_location:
            found_location = search_location.group(1)
            # Delete accent
            found_location = unidecode.unidecode(found_location)
            if any(word.lower() in found_location.lower() for word in CA_cities): 
                search_id = re.search('id\': (.+?),', str(tweet.user))
                if search_id:
                    found_id = search_id.group(1)
                    CA_ids.append(found_id)
    CAS_id.append(CA_ids)


TweepError: Twitter error response: status code = 429

In [73]:
CA_ids

12         Santa Maria de Gracia
42                      Arrecife
44    Las Palmas de Gran Canaria
52        Santa Cruz de Tenerife
Name: city, dtype: object

In [59]:
df_ids = pd.DataFrame(CAS_ids, columns = ["Andalusia", "Madrid", "Catalonia", "Basque_Country", "Canary_Islands"]) 

In [60]:
df_ids.to_csv("CAS_ids.csv")

In [40]:
tweets = tweepy.Cursor(api.user_timeline, id = "1084113752948199424").items(100)
for tweet in tweets:
    print(tweet.text)

@LaOtherF @maemiaelpatio Como??
@UfffXica @LaOtherF En mi tl está 😉
RT @mjrubiosa_: Yo en el amor. https://t.co/JkmBklEYcU
@LaOtherF @UfffXica para qué, ya está colgada como siempre.
Oye Línea Directa, no es nada personal, pero estamos hasta los cojones del Matías. Alguien tenía que decirlo.
RT @eldiarioes: Wyoming: "¿Si en lugar de mujeres se hubiera asesinado a 1.000 políticos, se hablaría de 'denuncias falsas'? https://t.co/Q…
@maemiaelpatio @Erika12505845 Eaaa!! Hasta luego https://t.co/CDMOctKxSV
@Erika12505845 Es el de echándome las manos a la cara
@maemiaelpatio Os tomáis mis tuits muy en serio y así no es, pero ya te lo explicaré cara a cara 😂😂
@maemiaelpatio @Erika12505845 Éramos pocos y...
@Erika12505845 Alaaaaaa!!! 🤦🏻‍♂️
@Erika12505845 Uyy q va, con lo que me gusta a mí besar en pleno acto sexual, por favor 😉
@Erika12505845 Maravilloso!!
@mebajolaluna 😘😘😘
Con vuestros ojos, tetas, culos, cuerpos sensuales y labios, aquí estoy, haciendo un puzzle y viendo si estáis follables 

In [23]:
search_location = re.search('blabalsdasd', str(tweet.user))

In [24]:
search_location

In [17]:
query = 'python'
max_tweets = 10
searched_tweets = [status.text for status in tweepy.Cursor(api.search, geocode=Andalusia + ",250km").items(max_tweets)]

In [18]:
searched_tweets

['lo mas interesante que me ha pasado hoy: he entrado en pokemon go y  han añadido una pokeparada al lado de mi puta casa soy feliz con poco',
 'vamos a vernos pero q nadie se entere',
 'RT @Barklysh: Cuando estoy triste veo este video y se me pasa https://t.co/d5AiCBUTIY',
 'RT @marinarguezh: Literalmente cuando he visto las dos opciones he hecho eso y me he replantado mi vida en 30 segundos https://t.co/w6OO6la…',
 'Así os pudrais @stradivarius',
 'RT @noeliaot2018: @auronplay La intención es lo que cuenta',
 'RT @IrethMarthu: Empezamos antes directo ;D gou!! https://t.co/TioiPZaWdm @espcomtriforce',
 '@juanan20burgui ort_coni_11@hotmail.com',
 'El viernes 14 de junio estaremos en la final de "Al filo" en el Parque San Sebastián de Ogíjares (Granada).… https://t.co/P3wFQClZyh',
 'RT @PetitAlba: Mi madre conmigo vs mi madre con mi novio https://t.co/kiEyVIS6mq']

In [65]:
str(a)

"User(_api=<tweepy.api.API object at 0x000001B2F752B0B8>, _json={'id': 725760504, 'id_str': '725760504', 'name': 'Daniel Castro', 'screen_name': 'DanielCL97', 'location': 'Los Llanos de Aridane', 'description': '⚽️', 'url': None, 'entities': {'description': {'urls': []}}, 'protected': False, 'followers_count': 553, 'friends_count': 700, 'listed_count': 0, 'created_at': 'Mon Jul 30 10:09:45 +0000 2012', 'favourites_count': 1227, 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'verified': False, 'statuses_count': 2856, 'lang': 'es', 'contributors_enabled': False, 'is_translator': False, 'is_translation_enabled': False, 'profile_background_color': '131516', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme14/bg.gif', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme14/bg.gif', 'profile_background_tile': True, 'profile_image_url': 'http://pbs.twimg.com/profile_images/1084976236944863234/IuAScNiN_normal.jpg', 'profile_image_

In [69]:
import re

text = 'L97\', \'location\': \'Los Llanos de Aridane\', '

m = re.search('location\':(.+?),', text)
if m:
    found = m.group(1)

In [70]:
found

" 'Los Llanos de Aridane'"

In [49]:
comunidades_tweets = []
for comunidad in comunidades:
    comunidad_tweet = []
    tweets = tweepy.Cursor(api.search, geocode = comunidad + ",150km").items(10)
    for tweet in tweets:
        comunidad_tweet.append(tweet.text)
    comunidades_tweets.append(comunidad_tweet)

In [50]:
comunidades_tweets[0]

['@Gonspliters https://t.co/Rt6p2bp56I',
 '@lestermillones Ahí tienes la foto!!!!',
 '@isabelsg_04: t voy a hacer un collar\nisabelmariasanch3 10 mins después: https://t.co/zmHXovVEbg',
 'Juego de monos. A favor.',
 'Ancestor the humankind https://t.co/UP7MdMClaw',
 'RT @Espball: elle https://t.co/cqywQeKDwR',
 '@martalarag pues mejor no cuentes los que te quedan',
 '@belsami2003 @emilioalamos @iesPSD_Alcorcon En los Centros publicos,generalmente, no llevan uniformes,pero el Conse… https://t.co/CPYN4EyPnj',
 '@Jon_Landini @MagiaPokemon la mierda esta xd https://t.co/zeQ14SjwKg',
 '@esa_abuela @onwonw1 Es bien... muy bien https://t.co/1jTh9PSbv9']

In [41]:
comunidades_tweets[0][10][0]

IndexError: list index out of range

In [42]:
len(comunidades_tweets[4])

10

In [46]:
comunidades_tweets_clean = []
for sentence in comunidades_tweets[0]:
    lower_case = sentence.lower()        # Convert to lower case
    words = lower_case.split()               # Split into words
    words = [w for w in words if not w in stopwords.words("spanish")]
    comunidades_tweets_clean += words

In [47]:
new_vals = Counter(comunidades_tweets_clean).most_common(10)

In [48]:
new_vals

[('rt', 5),
 ('echo', 1),
 ('mano', 1),
 ('@instagram', 1),
 ('https://t.co/dmecmf6ty7', 1),
 ('https://t.co/paitfw2oy7', 1),
 ('pf,', 1),
 ('paso', 1),
 ('agobiarme', 1),
 ('nota', 1)]